In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

D:\Anaconda\setup\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\setup\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Anaconda\setup\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [5]:
df = pd.read_csv('E:\\data\\nlp\\ner\\ner_dataset.csv', encoding = "ISO-8859-1")
#df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [6]:
#统计各字段为空的元素个数
print(df.isnull().sum())
#forward fill(用最近的有效字符来填充空白)
df = df.fillna(method='ffill')

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64


In [7]:
#查看每列所包含的不重复元素数:
df['Sentence #'].nunique(),df['Word'].nunique(),df['POS'].nunique(),df['Tag'].nunique()

(47959, 35178, 42, 17)

In [8]:
#统计每个Tag的数量，并降序排列
df.groupby('Tag').size().reset_index(name='num').sort_values('num',ascending=False)

,Tag,num
16,O,887908
2,B-geo,37644
7,B-tim,20333
5,B-org,20143
14,I-per,17251
6,B-per,16990
13,I-org,16784
3,B-gpe,15870
10,I-geo,7414
15,I-tim,6528


In [9]:
#引入sklearn中和CRF相关的包sklearn-crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [10]:
#数据预处理，将原始数据转换成指定格式
class SentenceTransform(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [11]:
#数据格式转换
t_result = SentenceTransform(df)
sent = t_result.get_next()
print(sent)

sentences = t_result.sentences
print(sentences[:3])

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]
[[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops

In [12]:
#特征提取
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [19]:
#生成训练和测试数据集
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
#print(X[:1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(y_train[:3])

[['O', 'B-gpe', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [20]:
#用CRF进行训练
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [21]:
#生成类别标签
classes = np.unique(df.Tag.values).tolist()
print(classes)
#去除类别"O"
classes.pop()
print(classes)

['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']
['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim']


In [22]:
#对模型效果进行预测
y_pred = crf.predict(X_test)
F1 = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)
print("F1:{0}".format(F1))

F1:0.8512087682284608


In [23]:
print(metrics.flat_classification_report(y_test, y_pred, labels = classes))

             precision    recall  f1-score   support

      B-art       0.45      0.12      0.19       111
      B-eve       0.47      0.37      0.41        79
      B-geo       0.86      0.91      0.89      9480
      B-gpe       0.97      0.94      0.96      3987
      B-nat       0.85      0.48      0.62        58
      B-org       0.80      0.74      0.77      5041
      B-per       0.85      0.83      0.84      4283
      B-tim       0.92      0.88      0.90      5240
      I-art       0.17      0.05      0.08        83
      I-eve       0.36      0.29      0.32        73
      I-geo       0.82      0.80      0.81      1931
      I-gpe       0.94      0.62      0.74        52
      I-nat       0.75      0.47      0.58        19
      I-org       0.81      0.80      0.81      4284
      I-per       0.85      0.90      0.87      4349
      I-tim       0.83      0.74      0.78      1673

avg / total       0.86      0.85      0.85     40743



In [24]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-nat  -> I-nat   7.123397
B-art  -> I-art   6.029598
I-art  -> I-art   5.938002
B-eve  -> I-eve   5.914369
I-eve  -> I-eve   5.640564
I-gpe  -> I-gpe   4.932274
I-tim  -> I-tim   4.588277
B-geo  -> I-geo   4.531746
I-org  -> I-org   4.411837
B-tim  -> I-tim   4.309793
B-org  -> I-org   4.176296
B-gpe  -> I-gpe   3.810566
I-geo  -> I-geo   3.647574
O      -> O       3.573804
B-per  -> I-per   3.560070
I-nat  -> I-nat   3.312426
I-per  -> I-per   2.715547
O      -> B-per   1.585798
O      -> B-tim   1.506052
I-geo  -> B-art   1.251252

Top unlikely transitions:
B-tim  -> I-org   -4.118340
B-geo  -> B-geo   -4.165929
I-per  -> I-org   -4.192177
B-per  -> I-org   -4.459603
B-tim  -> B-tim   -4.482683
B-gpe  -> I-per   -4.581899
O      -> I-art   -4.804808
B-gpe  -> I-geo   -4.906594
I-org  -> I-per   -4.908821
B-geo  -> I-per   -4.991151
B-geo  -> I-org   -5.113602
B-org  -> I-per   -5.313064
B-gpe  -> I-org   -5.609349
I-per  -> B-per   -6.042720
B-gpe  -> B-gpe  

In [25]:
import eli5

eli5.show_weights(crf, top=10)

D:\Anaconda\setup\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.574,0.035,-4.805,0.554,-3.781,0.912,-7.657,-0.164,-3.646,0.51,-2.27,0.808,-7.384,1.586,-6.592,1.506,-7.163
B-art,-0.236,0.0,6.03,0.0,-0.213,-0.816,-1.342,-1.397,-0.653,0.0,0.0,0.0,-2.214,-1.299,-2.235,0.001,-1.332
I-art,-0.293,-0.46,5.938,0.0,0.0,-0.238,-1.239,-0.84,-0.299,0.0,0.0,-1.093,-1.77,-1.005,-2.11,-0.981,-1.088
B-eve,-0.553,0.0,-0.274,-0.139,5.914,-1.211,-1.255,-1.188,-0.393,-0.356,0.0,-1.554,-1.838,-1.973,-1.961,0.352,-1.435
I-eve,0.18,0.0,0.0,-2.128,5.641,-0.935,-0.898,-0.408,0.0,0.0,0.0,-0.906,-1.33,-1.322,-1.378,-1.824,-1.304
B-geo,0.266,0.656,-2.33,-1.173,-2.149,-4.166,4.532,-0.119,-4.064,-0.392,-0.621,-0.524,-5.114,-1.685,-4.991,1.222,-3.887
I-geo,-0.009,1.251,-1.402,-0.358,-1.242,-3.082,3.648,-1.95,-2.4,0.0,-0.45,-0.662,-3.861,-1.056,-3.729,0.446,-2.941
B-gpe,0.537,-1.409,-2.08,-1.288,-2.477,-0.142,-4.907,-6.237,3.811,-0.648,-0.891,0.847,-5.609,-0.609,-4.582,-0.426,-3.476
I-gpe,0.158,0.0,0.0,0.0,0.0,-0.268,-1.286,-0.989,4.932,0.0,0.0,-1.993,-1.328,-0.276,-1.177,-0.711,-0.73
B-nat,-0.478,0.0,0.0,0.0,0.0,-0.409,-0.397,-0.749,0.0,-0.336,7.123,-0.293,-0.886,-0.763,-1.579,0.0,-0.243
